In [1]:
import time
import os
import cv2

from lib.file.TIF import *
from lib.analysis.ROI_identification import *
from lib.file.ROI_writer import ROIFileWriter
from lib.file.ROI_reader import ROIFileReader
from lib.file.TSV import RegionExporter
from lib.analysis.align import ImageAlign
from lib.file.DAT import TracesDAT
from lib.trace import Tracer
from lib.utilities import *
from lib.analysis.laminar_dist import *

In [3]:
data_dir = "C:/Users/jjudge3/Desktop/Data/Kate's/ROI_dist/"
data_dir = "G:/.shortcut-targets-by-id/1agT5mnPWaZdQIB8nNNVWaVbqpbYNGxcB/"
corners_file_prefix_layer = 'corners_layer'
corners_file_prefix_barrel = 'corners_barrel'
electrode_file_prefix = 'electrode'
rois_file_prefix = 'rois' # e.g. 'rois_01_to_09.dat'
snr_file_prefix = 'SNR'

image_data = {}
n_plots = 5  # how many to show

for subdir, dirs, files in os.walk(data_dir):
    print(subdir, dirs, files)
    for filename in files:
        if filename.split(".")[-1].endswith("_snr"):
            name_parse = filename[:8]  # of the format 07_01_01
            try:
                slic, loc, rec = [int(x) for x in name_parse.split('_')]
            except ValueError:
                continue
            if slic not in image_data:
                image_data[slic] = {}
            if loc not in image_data[slic]:
                image_data[slic][loc] = {}
            snr = np.loadtxt(data_dir + filename,
                             delimiter='\t',
                             usecols=(1,))
            width = int(np.sqrt(snr.shape[0]))
            snr = snr.reshape((width, width))

            print("slice", slic, ", rec", rec)

            # open corners, interlaminar, 2 points p1, p2 to define the edge along which to measure
            la_file = data_dir + corners_file_prefix_layer + '.dat'
            lines = None
            if os.file.exists(la_file)
                with open(la_file, 'r') as f:
                    lines = f.readlines() 
                corners = [int(x) for x in lines[4:]] # the last 4 lines are diode numbers of corners
                layer_axes = LayerAxes(corners)
                laminar_axis, laminar_axis_2 = layer_axes.get_layer_axes()
                print("laminar axis unit vectors:", laminar_axis.get_unit_vector(), 
                      laminar_axis_2.get_unit_vector())
                
            # same, but if we are measuring barrel (intralaminar) now
            la_file = data_dir + corners_file_prefix_barrel + '.dat'
            lines = None
            if os.file.exists(la_file)
                with open(la_file, 'r') as f:
                    lines = f.readlines() 
                corners = [int(x) for x in lines[4:]] # the last 4 lines are diode numbers of corners
                layer_axes = LayerAxes(corners)
                laminar_axis, laminar_axis_2 = layer_axes.get_layer_axes()
                print("laminar axis unit vectors:", laminar_axis.get_unit_vector(), 
                      laminar_axis_2.get_unit_vector())

            # open all rois as lists of diode numbers
            rois_file = data_dir + rois_file_prefix + suffix + '.dat'
            rois = ROIFileReader(rois_file).get_roi_list()
            rois = [LaminarROI(r, img_width=width) for r in rois]
            roi_centers = [r.get_center() for r in rois]
            roi_centers = [[round(r[0], 2), round(r[1], 2)] for r in roi_centers]
            print("ROI centers:", roi_centers)

            # open stim point roi as a single integer (its diode number) in variable stim_pt
            sp_file = data_dir + electrode_file_prefix + '.dat'
            with open(sp_file, 'r') as f:
                lines = f.readlines()
            stim_pt = int(lines[-1]) # last line is always electrode location
            aux_obj = LaminarROI([stim_pt]).get_points()
            stim_pt = aux_obj[0]  # should be a list of len 2, representing px location [x, y]
            print("Stim point:", stim_pt)

            # run laminar dist computation
            laminar_distances = LaminarDistance(laminar_axis, rois, stim_pt).compute_laminar_distances()
            laminar_distances = [round(ld, 2) for ld in laminar_distances]
            print('laminar_distances:', laminar_distances)  # a list of integers with same indexing as rois

            # run laminar dist again with the other axis
            laminar_distances_2 = LaminarDistance(laminar_axis_2, rois, stim_pt).compute_laminar_distances()
            laminar_distances_2 = [round(ld, 2) for ld in laminar_distances_2]
            print('laminar_distances:', laminar_distances_2)  # a list of integers with same indexing as rois

            image_data[slic][loc][rec] = {
                'snr': snr,
                'filename': filename,
                'laminar_distances': laminar_distances}

            # Now that the data is stored, show intermediate computations in plot
            if n_plots > 0:
                lines = [laminar_axis.get_line_repr(),
                         laminar_axis_2.get_line_repr()]
                line_colors = ['red', 'yellow']
                linewidths = [4,4,4,4]
                other_colors = ['red']

                # laminar aux lines
                uv = laminar_axis.get_unit_vector()
                for i in range(len(laminar_distances)):
                    ld = laminar_distances[i]
                    jiggle = i - int(len(laminar_distances) / 2)
                    stim_proxy_pt = [stim_pt[0], stim_pt[1] - jiggle]
                    measure_vector = [uv[0] * ld + stim_proxy_pt[0], uv[1] * ld + stim_proxy_pt[1]]
                    lines.append([measure_vector, stim_proxy_pt])
                    line_colors.append(other_colors[i % len(other_colors)])
                    linewidths.append(2)

                LaminarVisualization(snr, 
                                     stim_pt, 
                                     roi_centers, 
                                     layer_axes.get_corners(),
                                     lines,
                                     line_colors,
                                     linewidths,
                                     save_dir=subdir)
                n_plots -= 1


G:/.shortcut-targets-by-id/1agT5mnPWaZdQIB8nNNVWaVbqpbYNGxcB/ ['2020-12-28'] []
G:/.shortcut-targets-by-id/1agT5mnPWaZdQIB8nNNVWaVbqpbYNGxcB/2020-12-28 ['01-01-05_notUsable', '02-01-01_notUsable', '09-01-01_notUsable', '05-01-02_aka_part2_02-01-02_notUsable', '08-01-04_aka_part2_05-01-04_L4_interlaminar', '10-01-01_aka_part2_07-01-01_L4_interlaminar', '07-01-01_aka_part2_04-01-01_notUsable', '06-01-01_aka_part2_03-01-01_L4_interlaminar', '03-01-08_aka_02-01-08_L23_intralaminar', '04-01-01_aka_03-01-01_L4_interlaminar', '11-01-06_aka_part2_08-01-06_L4_interlaminar'] []
G:/.shortcut-targets-by-id/1agT5mnPWaZdQIB8nNNVWaVbqpbYNGxcB/2020-12-28\01-01-05_notUsable [] ['Not_Usable.txt', 'desktop.ini']
G:/.shortcut-targets-by-id/1agT5mnPWaZdQIB8nNNVWaVbqpbYNGxcB/2020-12-28\02-01-01_notUsable [] ['Not_Usable.txt', 'desktop.ini']
G:/.shortcut-targets-by-id/1agT5mnPWaZdQIB8nNNVWaVbqpbYNGxcB/2020-12-28\09-01-01_notUsable [] ['Not_Usable.txt', 'desktop.ini']
G:/.shortcut-targets-by-id/1agT5mnPWaZdQI

G:/.shortcut-targets-by-id/1agT5mnPWaZdQIB8nNNVWaVbqpbYNGxcB/2020-12-28\04-01-01_aka_03-01-01_L4_interlaminar\Old\Older ['test to see if can make image fit in photoZ', 'choosing rois with single pixels', 'choosing rois with bin 3'] ['desktop.ini']
G:/.shortcut-targets-by-id/1agT5mnPWaZdQIB8nNNVWaVbqpbYNGxcB/2020-12-28\04-01-01_aka_03-01-01_L4_interlaminar\Old\Older\test to see if can make image fit in photoZ [] ['test_2020-12-18_04-01-01_choosing_trajectories_L4_intralaminar.png', 'test_2020-12-18_04-01-01_choosing_trajectories_L4_intralaminar.jpg', 'test_2020-12-18_04-01-01_choosing_trajectories_L4_intralaminar.bmp', 'test.png', 'test_2020-12-18_04-01-01_choosing_trajectories_L4_intralaminar.psd', 'test2.png', 'test2.bmp', 'test3_original_jpg_size.jpg', 'test3_save_left_window.png', 'test3_resized.psd', 'test3_resized.jpg', 'test3_resized.bmp', 'test3_original_jpg_size.bmp', 'test3_original_jpg_size_plus200onL.jpg', 'test3_original_jpg_size_plus250onL.jpg', 'test3_original_jpg_size_pl